## distribute data

### data info

In [ ]:
CISIDATA = '../../../../CISI/CISI.ALL'
import re
IDMarker = re.compile('(\.I.)')
allMarkers = re.compile('(\.[ITABWX] )')

### queries info

In [ ]:
CISIQUERY = '../../../../CISI/CISI.QRY'
CISIQRELS = '../../../../CISI/CISI.REL'
import re
queryMarkers = re.compile('(\.[ITAWB] )')

In [ ]:
def getData(PATH, marker):
    """get the data from the file and split it by ID"""
    with open(PATH, 'r') as f:
        t = f.read().replace('\n', ' ')
        lines = re.split(marker, t)
        lines.pop(0)
    return lines

### converte CISI.ALL

In [ ]:
cisiData = getData(CISIDATA, allMarkers)

In [ ]:
import pandas as pd
dataFrame = pd.DataFrame(columns=['.I','.T','.A','.B','.W','.X'])
seriesDict:dict = {
    '.I': None,
    '.T': None,
    '.A': None,
    '.B': None,
    '.W': None,
    '.X': None
}
seriesData = seriesDict.copy()
notTheFirst = False
for i in range(0, len(cisiData), 2):
    if (notTheFirst and cisiData[i].strip() == '.I'):
        dataFrame = dataFrame.append(seriesData, ignore_index=True)
        seriesData = seriesDict.copy()
    
    seriesData[cisiData[i].strip()] = cisiData[i+1].strip()
    notTheFirst = True
dataFrame = dataFrame.append(seriesData, ignore_index=True)
dataFrame.head()

In [ ]:
dataFrame.to_csv('../../cisiData/cisiCsv.csv')

### converte query.text

In [ ]:
cisiQuery = getData(CISIQUERY, queryMarkers)

In [ ]:
import pandas as pd
qDataFrame = pd.DataFrame(columns=['.I','.T','.A','.W','.B'])
seriesDict:dict = {
    '.I': None,
    '.T': None,
    '.A': None,
    '.W': None,
    '.B': None
}
seriesData = seriesDict.copy()
notTheFirst = False
for i in range(0, len(cisiQuery), 2):
    if (notTheFirst and cisiQuery[i].strip() == '.I'):
        qDataFrame = qDataFrame.append(seriesData, ignore_index=True)
        seriesData = seriesDict.copy()
    
    seriesData[cisiQuery[i].strip()] = cisiQuery[i+1].strip()
    notTheFirst = True
qDataFrame = qDataFrame.append(seriesData, ignore_index=True)
qDataFrame.head()

In [ ]:
qDataFrame.to_csv('../../cisiData/cisiQueryCsv.csv')

### converte qrels.text

In [ ]:
import pandas as pd

def getRles(path):
    with open(path, 'r') as f:
        global qrlesList
        qrlesList = f.read().split('\n')
        return qrlesList

qrelsData = getRles(CISIQRELS)
qrelsFrame = pd.DataFrame(columns=['.I', 'data'])
seriesDict:dict = {'.I':None, 'data':None}
seriesData = seriesDict.copy()
for i in qrelsData:
    try:
        element = i.split()
        seriesData['.I'] = int(element[0])
        seriesData['data'] = int(element[1])
        qrelsFrame = qrelsFrame.append(seriesData, ignore_index=True)
    except:
        pass
qrelsFrame.head()

In [ ]:
qrelsFrame.to_csv('../../cisiData/cisiQRels.csv')

## clean preproccesing

### CISI

In [ ]:
import pandas as pd
df:pd.DataFrame = pd.read_csv('../../cisiData/cisiCsv.csv', index_col=[0])
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
print(df.loc[:,'.T'].isnull().value_counts(), end='\n\n')
print(df.loc[:,'.W'].isnull().value_counts(), end='\n\n')
print(df.loc[:,'.A'].isnull().value_counts())

In [ ]:
df.fillna('', inplace=True)

In [ ]:
df.duplicated().value_counts()

In [ ]:
df.to_csv('../../cisiData/cisiDataCleaned.csv')

#### methods

##### lowercase

In [ ]:
def toLower(text):
    return text.lower()

##### Numbers to words

In [ ]:
import inflect
p = inflect.engine()

import re
reg = r'([0-9]+)'

def isFLoat(strNum):
    try:
        float(strNum)
        return True
    except:
        return False


def converteNumbers(text):
    tempText = text.split()
    newText = []
    for word in tempText:
        tempList = re.split(reg,word)
        for miniWord in tempList:
            if miniWord.isdigit() or isFLoat(miniWord):
                temp = p.number_to_words(miniWord)
                newText.append(removePunctuation(temp))
            else:
                newText.append(miniWord)        
    tempText = ' '.join(newText)
    return tempText
    # return text

##### remove punctuation

In [ ]:
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
def removePunctuation(text):
    global translator
    return text.translate(translator)


##### remove whitespaces

In [ ]:
def removeWhiteSpace(text):
    return " ".join(text.split())

##### remove stop words

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist



def removeStopWords(text):
    sw = set(stopwords.words("english"))
    wt = word_tokenize(text)
    filteredText = [word for word in wt if word not in sw]
    return ' '.join(filteredText)
    # return text

##### calculate Frequency

In [ ]:
import numpy as np
def calcFreq(tokens):
    listOfTokens = tokens.split()
    fdist = FreqDist(word for word in listOfTokens)
    fdistKeys = np.array(list(fdist.keys()))
    freqOfWords = [fdist.freq(x) for x in fdistKeys]

    q1, q3 = np.percentile(freqOfWords, [25, 75])
    IQR = q3 - q1
    AVG = np.mean(freqOfWords)
    AvgRelValue = round(AVG * len(listOfTokens))
    Q1RelValue = round(q1* len(listOfTokens))
    Q3RelValue = round(q3 * len(listOfTokens))
    st = ' '.join(listOfTokens)

    for i in range(0,len(freqOfWords)):
        if freqOfWords[i] < q1 - 1.5*IQR:
            wordRelValue = round(freqOfWords[i] * len(listOfTokens))
            sub = Q1RelValue - wordRelValue
            word = fdistKeys[i]
            stForAppend = (' '+word+' ') * sub
            st = st + stForAppend

        if freqOfWords[i] > q3 + 1.5*IQR:
            wordRelValue = round(freqOfWords[i] * len(listOfTokens))
            sub = wordRelValue - Q3RelValue
            word = fdistKeys[i]
            st = st.replace(word, '', sub)
    
    return removeWhiteSpace(st)



In [ ]:
st = '''note pseudo mathemat relev taube recent number articl book report deal inform system e document retriev system advanc doctrin system evalu term degre percentag relev provid although seem littl agreement relev mean doubt quantifi nevertheless grow agreement fix formal relationship exist relev recal perform system thu find literatur frankli subject notion relev report individu user equat curv mathemat formul presum provid numer measur recal relev characterist inform system phenomenon shift back forth admittedli subject non mathemat term equat term give mathemat valu mathemat definit ancient parallel discus probabl one cours legisl mean term depend alic point master user term hand use singl term document cover two distinct mean especi usag design secur accept doctrin attribut mathemat valid repres seriou situat mere careless ambigu'''
qt = '''problem concern make descript titl difficulti involv automat retriev articl approxim titl usual relev content articl titl'''
stest = 'saher fatima fatima fatima fatima fatima may may may may may may moh moh moh moh moh '
calcFreq(stest)

##### fill query Title with the most freq 

In [ ]:
import numpy as np
def addMostFreq(tokens):
    listOfTokens = tokens.split()
    fdist = FreqDist(word for word in listOfTokens)
    fdistKeys = np.array(list(fdist.keys()))
    freqOfWords = [fdist.freq(x) for x in fdistKeys]

    AVG = np.mean(freqOfWords)
    stlis = []

    for i in range(0,len(freqOfWords)):
        if freqOfWords[i] >= AVG:
            stlis.append(fdistKeys[i])

    
    return removeWhiteSpace(' '.join(stlis))



In [ ]:
# import numpy as np


# def checkOutLiers(lineOfPercent):
#     try:
#         q1 = np.percentile(lineOfPercent, 25, interpolation='midpoint')
#         q3 = np.percentile(lineOfPercent, 75, interpolation='midpoint')
#         IQR = q3 - q1
#         AVG = np.mean(lineOfPercent)
#         lineAfterRemoveOutLiers = []
#         for i in lineOfPercent:
#             if i < q1 - 1.5*IQR:
#                 lineAfterRemoveOutLiers.append(AVG)
#             elif i > q3 + 1.5*IQR:
#               lineAfterRemoveOutLiers.append(AVG)
#             else:
#              lineAfterRemoveOutLiers.append(i)

#         return lineAfterRemoveOutLiers
#     except:
#         return lineOfPercent



# def checkCoverage(queryTokens) -> list:
#     covDoc = []

#     qTokens = np.array(queryTokens)
#     for doc in wordsFreq:
#         WordsList = np.array(list(doc.keys()))
#         common = np.intersect1d(qTokens, WordsList)
#         freqOfWords = [doc.freq(x) for x in common]
#         common = checkOutLiers(freqOfWords)
#         try:
#             covDoc.append(sum(common))
#         except:
#             covDoc.append(0)
    
#     sortedList = np.array(covDoc).argsort(axis=0)[::-1]
#     return sortedList

# # checkCoverage(queryTokens=['saher', 'fatima', 'man', 'hello'])

##### stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
def stemWords(text):
    global stemmer
    wt = word_tokenize(text)
    stems = []
    for word in wt:
        temp = stemmer.stem(word)
        # if not temp == word:
        #     temp = correctWords(temp)
        stems.append(temp)
    return ' '.join(stems)
    # return text

##### lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import pos_tag, defaultdict

lemmatizer = WordNetLemmatizer()

tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV




def lemmatizeWords(text):
    # wt = word_tokenize(text)
    # lemmas = [lemmatizer.lemmatize(word, pos='a') for word in wt]
    # return ' '.join(lemmas)
    # return text

    tokens = word_tokenize(text)
    lmtzr = WordNetLemmatizer()
    lemmas = [lmtzr.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(tokens) ]
    return ' '.join(lemmas)
# lemmatizeWords('hard')

##### correcting

In [ ]:
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.corpus import words
correct_words = words.words()
incorrectWords = '''preliminari'''.split()
result = []
def correctWords(text):
    for word in text:
        try:
            temp = [(jaccard_distance(set(ngrams(word, 2)),
                                      set(ngrams(w, 2))),w)
                                      for w in correct_words if w[0] == word[0]]
            result.append(sorted(temp, key = lambda val:val[0])[0][1])
        except:
            pass
    return ' '.join(result)
# correctWords(incorrectWords)

#### cisi process

In [ ]:
import pandas as pd
import re

def TitlePreProcesse(t):
    tempText = toLower(t)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)

    return tempText

def abstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    tempText = calcFreq(tempText)

    return tempText

# i didn't do it yet on cisi (converte date to timestamp)
def publicationPreProcesse(p): 
    # tempText = p.replace('cisi ','')
    # return pd.to_datetime(tempText)
    return p
    
def authorPreProcesse(a):
    tempText = toLower(a)
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word))
    names = ' '.join(l)
    return names


In [ ]:
import pandas as pd
def preprocessedData(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame()
    seriesDict:dict = {} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            tempT = tempA = tempB = tempW = None
            if not dataFrame.loc[i, '.T'] == '':
                tempT = TitlePreProcesse(dataFrame.loc[i, '.T'])
            if not dataFrame.loc[i, '.A'] == '':
                tempA = authorPreProcesse(dataFrame.loc[i, '.A'])
            if not dataFrame.loc[i, '.B'] == '':
                tempB = publicationPreProcesse(dataFrame.loc[i, '.B'])
            if not dataFrame.loc[i, '.W'] == '':
                tempW = abstractPreProcesse(dataFrame.loc[i, '.W'])
            

            seriesData['.I'] = i+1
            seriesData['.T'] = tempT
            seriesData['.A'] = tempA
            seriesData['.B'] = tempB
            seriesData['.W'] = tempW
            
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [ ]:
import pandas as pd
data = pd.read_csv('../../cisiData/cisiDataCleaned.csv', index_col=[0])
data.fillna('', inplace=True)
data.head()

In [ ]:
processedDAta = preprocessedData(data)
processedDAta.head()

In [ ]:
processedDAta.to_csv('../../cisiData/cisiDataPreprocessed.csv')

#### query process

In [ ]:
import pandas as pd
import re


def qTitlePreProcesse(t):
    tempText = toLower(t)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)

    return tempText

def qAbstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    tempText = calcFreq(tempText)
    return tempText

def qAuthorPreProcesse(a):
    tempText = toLower(a)
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word))
    names = ' '.join(l)
    return names


# i didn't do it yet on cisi (converte date to timestamp)
def qPublicationPreProcesse(p):
    # tempText = p.replace('cisi ','')
    # return pd.to_datetime(tempText)
    return p


In [ ]:
import pandas as pd
def preprocesseQuery(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame() 
    seriesDict:dict = {} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            tempT = tempA = tempW = tempB = None
            if not dataFrame.loc[i, '.T'] == '':
                tempT = qTitlePreProcesse(dataFrame.loc[i, '.T'])
            if not dataFrame.loc[i, '.A'] == '':
                tempA = qAuthorPreProcesse(dataFrame.loc[i, '.A'])
            if not dataFrame.loc[i, '.W'] == '':
                tempW = qAbstractPreProcesse(dataFrame.loc[i, '.W'])
                if dataFrame.loc[i, '.T'] == '':
                    tempT = addMostFreq(tempW)
                    
            if not dataFrame.loc[i, '.B'] == '':
                tempB = qPublicationPreProcesse(dataFrame.loc[i, '.B'])


                
            seriesData['.I'] = i+1
            seriesData['.T'] = tempT
            seriesData['.A'] = tempA
            seriesData['.W'] = tempW
            seriesData['.B'] = tempB
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [ ]:
import pandas as pd
querydf = pd.read_csv('../../cisiData/cisiQueryCsv.csv', index_col=[0])
querydf.fillna('', inplace=True)
querydf.head()

In [ ]:
preprocessedQuery = preprocesseQuery(querydf)
preprocessedQuery.head()

In [ ]:
preprocessedQuery.to_csv('../../cisiData/cisiQueryPreprocessed.csv')

## indexing model

In [25]:
import pandas as pd
data1 = pd.read_csv('../../cisiData/cisiDataPreprocessed.csv', index_col=[0])
data1['.A'].notnull().value_counts()

True    1460
Name: .A, dtype: int64

In [26]:
import pandas as pd
data = pd.read_csv('../../cisiData/cisiDataPreprocessed.csv', index_col=[0])
data.fillna('', inplace=True)
data.head()

,.I,.T,.A,.B,.W
0,1.0,eighteen edit dewey decim classif,comaromi,,present studi decim classif publish eight six ...
1,2.0,use make technic librari,slater,,report analysi six thousand three hundr four u...
2,3.0,two kind power essay bibliograph control,wilson,,relationship organ organ inform inevit enter s...
3,4.0,system analysi univers librari final report re...,buckland,,new one thousand nine provok highli stimul exa...
4,5.0,librari manag game report research project,brophy,,although widespread last decad method number f...


In [27]:
#
# this is old cell for build index model with merge coulmns into one coulmn (not work now)
#

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import  cosine_similarity, linear_kernel


# tfidf = TfidfVectorizer()
# tfidfTable = tfidf.fit_transform(data['.W'])


In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline



transformer = FeatureUnion([
                  ('title_tfidf', 
                  Pipeline([
                    ('extract_field',
                              FunctionTransformer(lambda x: x['.T'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer(norm='l2' ,ngram_range=(1,2)))]))                              
                  ,('abstract_tfidf',
                 Pipeline([('extract_field',
                            FunctionTransformer(lambda x: x['.W'],
                                                  validate=False)),
                            ('tfidf',
                              TfidfVectorizer(norm='l1',ngram_range=(1,2)))]))
                ,('author_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['.A'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer(norm='l1'))]))
])
tfidfTable = transformer.fit_transform(data)
tfidfTable

<1460x77879 sparse matrix of type '<class 'numpy.float64'>'
	with 171911 stored elements in Compressed Sparse Row format>

In [ ]:
tfidfTable.toarray()[1000:20000]

In [29]:
import pandas as pd
preprocessedQuery = pd.read_csv('../../cisiData/cisiQueryPreprocessed.csv', index_col=[0])
preprocessedQuery.fillna('', inplace=True)
preprocessedQuery.head()

,.I,.T,.A,.W,.B
0,1.0,problem concern make descript difficulti invol...,,problem concern make descript difficulti invol...,
1,2.0,actual pertin data oppos refer entir articl re...,,actual pertin data oppos refer entir articl re...,
2,3.0,inform scienc give definit possibl,,inform scienc give definit possibl,
3,4.0,imag recognit method automat transform print t...,,imag recognit method automat transform print t...,
4,5.0,special train ordinari research businessmen ne...,,special train ordinari research businessmen ne...,


In [10]:

d = {}
d[1] = preprocessedQuery.loc[preprocessedQuery['.I'] == 1, ['.T', '.A', '.W']].to_dict()
d

{1: {'.T': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'},
  '.A': {0: ''},
  '.W': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'}}}

In [24]:

obj = {
    'bla':{
        'reslut':{
            1: {'.T': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'},
                '.A': {0: ''},
                '.W': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'}},

             2: {'.T': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'},
                 '.A': {0: ''},
                 '.W': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'}},

             3: {'.T': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'},
                 '.A': {0: ''},
                 '.W': {0: 'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'}}
        },
        'correcting': 'query string'
    }
}

evaluateObj = {
    'result':{
        'precision':0.005,
        'recall': 0.005,
        'MAP': 0.004,
        'MRR': 0.008
    },
    'n':10
}
obj['bla']['reslut'][1]['.T'][0]

'problem concern make descript difficulti involv automat retriev approxim usual relev content articl titl'

In [30]:
import numpy as np
def search(query,n:int):
    querytfidf = transformer.transform(query)

    cos = cosine_similarity(querytfidf,tfidfTable).flatten()
    resultList = cos.argsort(axis=0)[-n:][::-1]
    ids = []
    for i in resultList:
        ids.append(data.loc[i,'.I'])
        
    return ids
    return query['.W'].tolist()[0]

# ex query num 1 (too bad result)
search(pd.DataFrame(preprocessedQuery.loc[preprocessedQuery.index == 0,:]),10)

[1033.0, 958.0, 429.0, 90.0, 1054.0, 596.0, 550.0, 29.0, 486.0, 479.0]

In [31]:
t = 'note pseudo relev'
w = ' singl term document cover two distinct mean especi usag design secur accept doctrin attribut mathemat valid repres seriou situat mere careless ambigu'
a = 'taube'
test = pd.DataFrame([[t,w,a]], columns=['.T', '.W', '.A'])
search(test, 2)

[28.0, 447.0]

In [32]:
def queryingData(qDataFrame:pd.DataFrame, n):

    result = pd.DataFrame()

    for i in qDataFrame.index:
        try:
            resultDict:dict = {}
            tempList:list = search(pd.DataFrame(qDataFrame.loc[qDataFrame.index == i,:]), n)
            for id in range(1,n+1):
                resultDict[str(id)] = tempList[id - 1]
            result = result.append(resultDict, ignore_index=True)
        except:
            print(i)
            raise
    return result


In [33]:
queriesResult = queryingData(preprocessedQuery, 10)
queriesResult.head()

,1,2,3,4,5,6,7,8,9,10
0,1033.0,958.0,429.0,90.0,1054.0,596.0,550.0,29.0,486.0,479.0
1,1156.0,1158.0,1155.0,1138.0,532.0,552.0,462.0,1377.0,175.0,179.0
2,60.0,537.0,599.0,1027.0,554.0,572.0,1022.0,445.0,598.0,553.0
3,666.0,1280.0,1294.0,601.0,1105.0,643.0,565.0,72.0,1144.0,633.0
4,458.0,538.0,1307.0,1282.0,648.0,462.0,827.0,1054.0,826.0,319.0


## Evaluation

### precision

In [104]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score

def reSizeLists(l1:list, l2:list):
    '''resize lists to have the same len'''
    if len(l1) < len(l2):
        l2 = l2[0:len(l1)]
    while len(l1) > len(l2):
        l1 = l1[0:len(l2)]

    return l1, l2


def precWithoutOrder(l1:list,l2:list):
    ''' calculate precision witout orering'''
    try:
        return len(set(l1).intersection(set(l2))) / len(l2)
    except:
        return 0

def calcMAPrecisionAtK(resData:pd.DataFrame, qrelsData: pd.DataFrame):
    '''calcualte MAP'''
    precisionsAtK:list = []
    precisionAtK:float

    for i in resData.index:
        resArray = resData.loc[i].to_numpy()
        qresArray = qrelsData.loc[qrelsData['.I'] == i+1, 'data'].to_numpy()
        
        if len(qresArray) == 0: 
            continue

        resArray, qresArray = reSizeLists(resArray, qresArray)



        prec = precision_score(qresArray, resArray, average='micro')
        # prec = precWithoutOrder(qresArray, resArray)

        precisionsAtK.append(prec)

    precisionAtK = sum(precisionsAtK) / len(precisionsAtK)
    return precisionAtK

def calcMAPrecisionAtKOrder(resData:pd.DataFrame, qrelsData: pd.DataFrame):
    '''calcualte MAP'''
    precisionsAtK:list = []
    precisionAtK:float

    for i in resData.index:
        resArray = resData.loc[i].to_numpy()
        qresArray = qrelsData.loc[qrelsData['.I'] == i+1, 'data'].to_numpy()
        
        if len(qresArray) == 0: 
            continue

        resArray, qresArray = reSizeLists(resArray, qresArray)



        # prec = precision_score(qresArray, resArray, average='micro')
        prec = precWithoutOrder(qresArray, resArray)

        precisionsAtK.append(prec)

    precisionAtK = sum(precisionsAtK) / len(precisionsAtK)
    return precisionAtK

def calcAPrecisionAtK(resData:pd.DataFrame, qrelsData: pd.DataFrame):
    '''calcualte Average Precision'''
    precisionsAtK:list = []

    for i in resData.index:
        precisionOnQuery = []

        resArray = resData.loc[i].to_numpy()
        qresArray = qrelsData.loc[qrelsData['.I'] == i+1, 'data'].to_numpy()
        
        if len(qresArray) == 0: 
            continue
        resArray, qresArray = reSizeLists(resArray, qresArray)

        for lenI in range(0,len(qresArray)):

            tempRes:list = resArray[0:lenI+1].tolist()
            tempQRes:list = qresArray[0:lenI+1].tolist()
            precisionOnQuery.append(precision_score(tempQRes, tempRes, average='micro'))

        try:
            precisionsAtK.append(sum(precisionOnQuery) / len(precisionOnQuery))
        except ZeroDivisionError: 
            precisionsAtK.append(0)
    return precisionsAtK


In [62]:
import pandas as pd
qrelsFrame = pd.read_csv('../../cisiData/cisiQRels.csv', index_col=[0])
qrelsFrame.head()

,.I,data
0,1,28
1,1,35
2,1,38
3,1,42
4,1,43


In [105]:
# K = 10
print(calcMAPrecisionAtK(queriesResult, qrelsFrame))
print(calcMAPrecisionAtKOrder(queriesResult, qrelsFrame))

0.005263157894736842
0.05749269005847955


0.023830409356725146 l1 - qFreq - DFreq - just .W ngram = (1,2)
0.08249269005847952 .W l2 ngram = (1,2) .T l2 ngram = (1,2) fill - .A l1 - Freq - not order - q1 q3

In [ ]:
# K = 10
averagePrecision = calcAPrecisionAtK(queriesResult, qrelsFrame)
for i in range(0,len(averagePrecision), 2):
    print(i+1, end=' - '); print(averagePrecision[i])

#### Mean Reciprocal Rank

In [138]:
import numpy as np

resultDataFrame = pd.DataFrame(columns=['.I','data', 'rank'])


for row in queriesResult.index:
    temp = queriesResult.loc[row].to_list()
    ke = []
    rank = []
    for i in range(0,len(temp)):
        ke.append(row + 1)
        rank.append(i+1)
    lot = zip(ke,temp, rank)
    tempdata = pd.DataFrame(lot, columns=['.I','data', 'rank'])

    resultDataFrame = resultDataFrame.append(tempdata, ignore_index=True)




In [142]:
resultDataFrame[-3:]

,.I,data,rank
1117,112,1144.0,8
1118,112,495.0,9
1119,112,898.0,10


In [139]:
qrelsFrame.head()

,.I,data
0,1,28
1,1,35
2,1,38
3,1,42
4,1,43


In [143]:
def calcMeanReciprocalRank(resData:pd.DataFrame, qrelsData: pd.DataFrame):


    MAX_RANK = 100000

    hits = pd.merge(qrelsData, resData,
        on=[".I", "data"],
        how="left").fillna(MAX_RANK)

    mrr = (1 / hits.groupby('.I')['rank'].min()).mean()
    return mrr

In [144]:
calcMeanReciprocalRank(resultDataFrame, qrelsFrame)

0.42221963241436916

### i stoped here